In [ ]:
from datetime import datetime
from pydantic import BaseModel, PositiveInt
from typing import List, Literal
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import subprocess
from fastapi import FastAPI, Depends
import asyncio
from uvicorn import Server, Config

In [ ]:
class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: datetime | None = None

# Создание экземпляра модели
external_data = {
    'id': 123,
    'signup_ts': '2023-03-01 12:00',
    'name': 'Jane Doe'
}

user = User(**external_data)

# Вывод атрибутов модели
print(user.id)
print(user.name)
print(user.signup_ts)

# Вывод модели в словаре
print(user.dict())


123
Jane Doe
2023-03-01 12:00:00
{'id': 123, 'name': 'Jane Doe', 'signup_ts': datetime.datetime(2023, 3, 1, 12, 0)}


/tmp/ipykernel_15761/2755146756.py:21: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  print(user.dict())


In [ ]:
class Product(BaseModel):
    id: int
    name: str
    price: float
    ratings: List[PositiveInt] = []

# Создание экземпляра модели
product_data = {
    'id': 1,
    'name': 'Product A',
    'price': 19.99,
    'ratings': [5, 4, 5]
}

product = Product(**product_data)

# Вывод атрибутов модели
print(product.id)
print(product.name)
print(product.price)
print(product.ratings)


1
Product A
19.99
[5, 4, 5]


In [ ]:
class ColorModel(BaseModel):
    color: Literal['red', 'green', 'blue']

# Создание экземпляра модели
color_data = {
    'color': 'red'
}

color_model = ColorModel(**color_data)

# Вывод атрибута модели
print(color_model.color)

# Попытка создать экземпляр с невалидным цветом
try:
    invalid_color_data = {
        'color': 'yellow'
    }
    ColorModel(**invalid_color_data)
except Exception as e:
    print(e)


red
1 validation error for ColorModel
color
  Input should be 'red', 'green' or 'blue' [type=literal_error, input_value='yellow', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/literal_error


In [ ]:
class Address(BaseModel):
    street: str
    city: str

class User(BaseModel):
    id: int
    name: str
    address: Address
    signup_ts: datetime | None = None

# Создание экземпляра модели
user_data = {
    'id': 123,
    'name': 'John Doe',
    'address': {
        'street': 'Main St',
        'city': 'New York'
    },
    'signup_ts': '2023-01-01 12:00'
}

user = User(**user_data)

# Вывод атрибутов модели
print(user.id)
print(user.name)
print(user.address.street)
print(user.address.city)
print(user.signup_ts)


123
John Doe
Main St
New York
2023-01-01 12:00:00


In [ ]:
engine = create_engine('sqlite:///example.db')

# Создание базового класса для моделей
Base = declarative_base()

# Определение модели для таблицы
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)

# Создание таблицы в базе данных
Base.metadata.create_all(engine)

# Создание сессии для взаимодействия с базой данных
Session = sessionmaker(bind=engine)
session = Session()

# Добавление нового пользователя
new_user = User(name='John Doe', age=30)
session.add(new_user)
session.commit()

# Получение всех пользователей
users = session.query(User).all()
for user in users:
    print(f'ID: {user.id}, Name: {user.name}, Age: {user.age}')


ID: 1, Name: John Doe, Age: 31
ID: 2, Name: John Doe, Age: 30
ID: 3, Name: John Doe, Age: 30


/tmp/ipykernel_15761/3197048043.py:4: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [ ]:
# Вставка нового пользователя
new_user = User(name='Jane Doe', age=25)
session.add(new_user)
session.commit()

# Обновление существующего пользователя
user_to_update = session.query(User).filter_by(name='John Doe').first()
user_to_update.age = 31
session.commit()

# Удаление пользователя
user_to_delete = session.query(User).filter_by(name='Jane Doe').first()
session.delete(user_to_delete)
session.commit()


In [ ]:
# Получение пользователей с возрастом больше 30
users_over_30 = session.query(User).filter(User.age > 30).all()
for user in users_over_30:
    print(f'ID: {user.id}, Name: {user.name}, Age: {user.age}')

# Получение пользователей, отсортированных по имени
sorted_users = session.query(User).order_by(User.name).all()
for user in sorted_users:
    print(f'ID: {user.id}, Name: {user.name}, Age: {user.age}')


ID: 1, Name: John Doe, Age: 31
ID: 1, Name: John Doe, Age: 31
ID: 2, Name: John Doe, Age: 30
ID: 3, Name: John Doe, Age: 30


In [ ]:
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

In [ ]:
app = FastAPI()

@app.get("/users/")
def read_users(db: Session = Depends(get_db)):
    return db.query(User).all()

@app.post("/users/")
def create_user(name: str, age: int, db: Session = Depends(get_db)):
    db_user = User(name=name, age=age)
    db.add(db_user)
    db.commit()
    db.refresh(db_user)
    return db_user

@app.get("/users/{user_id}")
def read_user(user_id: int, db: Session = Depends(get_db)):
    return db.query(User).filter(User.id == user_id).first()

@app.put("/users/{user_id}")
def update_user(user_id: int, name: str, age: int, db: Session = Depends(get_db)):
    db_user = db.query(User).filter(User.id == user_id).first()
    db_user.name = name
    db_user.age = age
    db.commit()
    db.refresh(db_user)
    return db_user

@app.delete("/users/{user_id}")
def delete_user(user_id: int, db: Session = Depends(get_db)):
    db_user = db.query(User).filter(User.id == user_id).first()
    db.delete(db_user)
    db.commit()
    return {"message": f"User with ID {user_id} deleted"}

In [ ]:
config = Config(app=app, host="127.0.0.1", port=8000)

# Создайте сервер
server = Server(config)

await server.serve()

INFO:     Started server process [15761]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:57618 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:57618 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [15761]


Ниже код не отработает, тк **main:app** - это из **from main import app**, где **app** - это FastAPI. А файла **main.py** в этом юпитере нет.

In [ ]:
command = ["uvicorn", "main:app", "--reload"]
# command = ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8001", "--reload"]
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

Из консоли **uvicorn main:app --reload**, что позволит не перезапускать все при измении кода (**--reload**)

Теперь вы можете взаимодействовать с вашим API через Swagger UI по адресу http://127.0.0.1:8000/docs.